In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"seeranaveensai","key":"72f23fe4f950d6b80a7180225cb0d7d6"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
  0% 0.00/66.0M [00:00<?, ?B/s]
100% 66.0M/66.0M [00:00<00:00, 1.20GB/s]


In [6]:
!unzip creditcardfraud.zip

Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [7]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/creditcard.csv")

# Basic info
print("Shape of dataset:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nClass distribution:\n", df['Class'].value_counts())

# Preview
df.head()

Shape of dataset: (284807, 31)

Columns: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']

Class distribution:
 Class
0    284315
1       492
Name: count, dtype: int64


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
from sklearn.preprocessing import StandardScaler

# Scale Time and Amount
df['Time'] = StandardScaler().fit_transform(df['Time'].values.reshape(-1,1))
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1))

# Features & labels
X = df.drop('Class', axis=1)
y = df['Class']

print("X shape:", X.shape, " y shape:", y.shape)


X shape: (284807, 30)  y shape: (284807,)


In [9]:
# Features & labels
X = df.drop('Class', axis=1)
y = df['Class']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Train size:", X_train.shape, " Test size:", X_test.shape)


Train size: (227845, 30)  Test size: (56962, 30)


In [11]:
from sklearn.ensemble import RandomForestClassifier

# Train a small random forest (lightweight for PoC)
clf = RandomForestClassifier(n_estimators=50, random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)


In [12]:
from sklearn.metrics import classification_report, confusion_matrix

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[56859     5]
 [   18    80]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.94      0.82      0.87        98

    accuracy                           1.00     56962
   macro avg       0.97      0.91      0.94     56962
weighted avg       1.00      1.00      1.00     56962



In [13]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
weights = compute_class_weight(class_weight='balanced',
                               classes=np.unique(y_train),
                               y=y_train)

class_weights = {0: weights[0], 1: weights[1]}
print("Class Weights:", class_weights)


Class Weights: {0: np.float64(0.5008661206149896), 1: np.float64(289.14340101522845)}


In [15]:
import time

def transaction_stream(X, y, model, n=20):
    for i in range(n):  # stream only first 20 for demo
        txn = X[i].reshape(1, -1)
        pred = model.predict(txn)[0]
        actual = y[i]
        print(f"Txn {i+1}: Predicted={'FRAUD' if pred==1 else 'NORMAL'} | Actual={'FRAUD' if actual==1 else 'NORMAL'}")
        time.sleep(1)  # delay for real-time effect

transaction_stream(X_test.values, y_test.values, rf, n=20)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Txn 1: Predicted=NORMAL | Actual=NORMAL


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Txn 2: Predicted=NORMAL | Actual=NORMAL


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


...

Txn 18: Predicted=NORMAL | Actual=NORMAL


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Txn 19: Predicted=NORMAL | Actual=NORMAL


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Txn 20: Predicted=NORMAL | Actual=NORMAL


In [16]:
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE


In [17]:
# Apply SMOTE only on training data
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)

print("Original training set:", X_train.shape, y_train.value_counts().to_dict())
print("Resampled training set:", X_train_resampled.shape, y_train_resampled.value_counts().to_dict())


Original training set: (227845, 30) {0: 227451, 1: 394}
Resampled training set: (454902, 30) {0: 227451, 1: 227451}


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

rf_smote = RandomForestClassifier(n_estimators=100, random_state=42)
rf_smote.fit(X_train_resampled, y_train_resampled)

# Test on original test set
y_pred_smote = rf_smote.predict(X_test)

print("Classification Report (with SMOTE):\n")
print(classification_report(y_test, y_pred_smote, digits=4))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_smote))


Classification Report (with SMOTE):

              precision    recall  f1-score   support

           0     0.9997    0.9997    0.9997     56864
           1     0.8454    0.8367    0.8410        98

    accuracy                         0.9995     56962
   macro avg     0.9225    0.9182    0.9204     56962
weighted avg     0.9995    0.9995    0.9995     56962


Confusion Matrix:
 [[56849    15]
 [   16    82]]


In [19]:
# Choose the range (example: rows 65 to 100)
start = 65
end = 100

# Select samples from the given range
X_range = X_test.iloc[start:end]
y_range = y_test.iloc[start:end]

# Predictions
y_pred_range = rf_smote.predict(X_range)
y_proba_range = rf_smote.predict_proba(X_range)[:,1]

# Results DataFrame
results_range = pd.DataFrame({
    "Actual": y_range.values,
    "Predicted": y_pred_range,
    "Fraud_Probability": y_proba_range
}, index=X_range.index)

print(f"Predictions for test samples {start} to {end}:\n")
print(results_range)


Predictions for test samples 65 to 100:

        Actual  Predicted  Fraud_Probability
20971        0          0               0.00
158895       0          0               0.00
276003       0          0               0.00
28607        0          0               0.00
237933       0          0               0.00
152514       0          0               0.00
248114       0          0               0.00
6263         0          0               0.00
83265        0          0               0.00
35776        0          0               0.00
127082       0          0               0.00
281064       0          0               0.00
253147       0          0               0.00
201369       0          0               0.00
45223        0          0               0.00
230172       0          0               0.01
83362        0          0               0.00
162284       0          0               0.00
256823       0          0               0.00
74545        0          0               0.00
92940        0

In [21]:
# Get all fraud rows in test set
fraud_samples = X_test[y_test == 1]
fraud_labels = y_test[y_test == 1]

# Predict on these fraud samples
y_pred_fraud = rf_smote.predict(fraud_samples)
y_proba_fraud = rf_smote.predict_proba(fraud_samples)[:,1]

fraud_results = pd.DataFrame({
    "Actual": fraud_labels.values,
    "Predicted": y_pred_fraud,
    "Fraud_Probability": y_proba_fraud
}, index=fraud_samples.index)

print("Predictions only for FRAUD transactions:\n")
print(fraud_results.head(20))  # show first 20 frauds


Predictions only for FRAUD transactions:

        Actual  Predicted  Fraud_Probability
77348        1          1               0.98
102442       1          1               1.00
119781       1          1               0.83
48094        1          1               0.73
42958        1          1               0.96
52584        1          1               1.00
204064       1          1               0.95
151519       1          1               1.00
203324       1          1               0.96
141260       1          1               1.00
70589        1          1               0.55
14170        1          1               0.63
222133       1          1               0.98
153835       1          1               1.00
14197        1          1               0.87
229712       1          1               1.00
15810        1          1               1.00
8972         1          0               0.38
152019       1          1               0.51
30100        1          1               0.97


In [22]:
# Sample 10 fraud + 10 normal
fraud_sample = fraud_samples.sample(10, random_state=42)
normal_sample = X_test[y_test == 0].sample(10, random_state=42)

combined_X = pd.concat([fraud_sample, normal_sample])
combined_y = y_test.loc[combined_X.index]

# Predict
y_pred_combined = rf_smote.predict(combined_X)
y_proba_combined = rf_smote.predict_proba(combined_X)[:,1]

combined_results = pd.DataFrame({
    "Actual": combined_y.values,
    "Predicted": y_pred_combined,
    "Fraud_Probability": y_proba_combined
}, index=combined_X.index)

print("Comparison of FRAUD vs NORMAL transactions:\n")
print(combined_results)


Comparison of FRAUD vs NORMAL transactions:

        Actual  Predicted  Fraud_Probability
42528        1          1               0.92
149577       1          0               0.35
119714       1          0               0.01
152019       1          1               0.51
150663       1          1               1.00
106998       1          1               1.00
105178       1          1               0.79
64411        1          1               1.00
70589        1          1               0.55
77348        1          1               0.98
197288       0          0               0.00
175745       0          0               0.00
29594        0          0               0.00
195743       0          0               0.01
234355       0          0               0.00
201844       0          0               0.02
5560         0          0               0.00
170199       0          0               0.00
98802        0          0               0.00
267610       0          0               0.00


In [23]:
import time

# Take 10 normal + 10 fraud samples for streaming demo
normal_samples = X_test[y_test == 0].sample(10, random_state=42)
fraud_samples  = X_test[y_test == 1].sample(10, random_state=42)

# Concatenate and shuffle
stream_data = pd.concat([normal_samples, fraud_samples]).sample(frac=1, random_state=42)
stream_labels = y_test.loc[stream_data.index]

print("🔴 Streaming fraud detection demo...\n")

for i, (idx, row) in enumerate(stream_data.iterrows()):
    txn = row.values.reshape(1, -1)
    pred = rf_smote.predict(txn)[0]
    proba = rf_smote.predict_proba(txn)[0][1]

    # Print result clearly
    status = "🚨 FRAUD" if pred == 1 else "✅ Normal"
    print(f"Txn {i+1} | Actual: {stream_labels.loc[idx]} | Predicted: {pred} ({status}) | Fraud Probability: {proba:.4f}")

    time.sleep(0.5)


🔴 Streaming fraud detection demo...

Txn 1 | Actual: 0 | Predicted: 0 (✅ Normal) | Fraud Probability: 0.0000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Txn 2 | Actual: 1 | Predicted: 1 (🚨 FRAUD) | Fraud Probability: 1.0000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Txn 3 | Actual: 1 | Predicted: 1 (🚨 FRAUD) | Fraud Probability: 1.0000


...

Txn 18 | Actual: 1 | Predicted: 1 (🚨 FRAUD) | Fraud Probability: 0.9200


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Txn 19 | Actual: 1 | Predicted: 1 (🚨 FRAUD) | Fraud Probability: 1.0000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Txn 20 | Actual: 0 | Predicted: 0 (✅ Normal) | Fraud Probability: 0.0000
